This notebook is starting point for Home Credit Default Risk Kaggle Problem Solution
Primary objective of this notbook is to load data from /input.nosync folder and perform basic EDA.
Detailed modelling and prediction should be done in separate notebook based on EDA results that we get here

In [1]:
# all the imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import pprint
import copy

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,Imputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

/Users/gauravdesai/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/gauravdesai/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Notebook lavel configurations
warnings.filterwarnings('ignore')
pp = pprint.PrettyPrinter(indent=4)

In [3]:
# List input files, if not available please download from below url inot a /input.nosync folder
input_dir = 'input.nosync'
input_files = os.listdir(input_dir)
if input_files is None or len(input_files) < 11 :
    raise Exception('You do not have all the files in {} directory'.format(input_dir))

print('You have all the input files listed below')
pp.pprint(input_files)

You have all the input files listed below
[   'application_test.csv',
    '.DS_Store',
    'HomeCredit_columns_description.csv',
    'POS_CASH_balance.csv',
    'credit_card_balance.csv',
    'installments_payments.csv',
    'application_train.csv',
    'bureau.csv',
    'previous_application.csv',
    'bureau_balance.csv',
    'sample_submission.csv']


Read in application_train and application test, note any changes to train need to be made to test as well

In [4]:
# Read application_train.csv
app_train = pd.read_csv(input_dir+'/application_train.csv')
#app_test = pd.read_csv(input_dir+'/application_test.csv')
print('Training data shape: ', app_train.shape)
#print('Test data shape: ', app_test.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


Now lets crate test data by splitting training set

In [5]:
app_train_labels = app_train['TARGET']
app_train=app_train.drop('TARGET', axis=1)

# remove SK_ID_CURR for now since this column does not give any info about defaults
app_train=app_train.drop('SK_ID_CURR', axis=1)


app_train_data, app_test_data, train_labels, test_labels = train_test_split(
    app_train,app_train_labels,test_size = .2, random_state = 23 )

print('Training data shape: ', app_train_data.shape, train_labels.shape)
print('Test data shape: ', app_test_data.shape, test_labels.shape)

Training data shape:  (246008, 120) (246008,)
Test data shape:  (61503, 120) (61503,)


Convert categorical columns to numeric using One Hot Encoding

In [6]:
app_train_data = pd.get_dummies(app_train_data)
app_test_data = pd.get_dummies(app_test_data)

print('Training Features shape: ', app_train_data.shape)
print('Testing Features shape: ', app_test_data.shape)

Training Features shape:  (246008, 244)
Testing Features shape:  (61503, 243)


Align train and test so clumns match

In [7]:
app_train_data, app_test_data = app_train_data.align(app_test_data, join = 'inner', axis = 1)

In [8]:
print(app_train_data.shape)
print(app_test_data.shape)

(246008, 243)
(61503, 243)


Impute missing values, using the column mean

In [9]:
imputer = Imputer(strategy = 'median')

app_train_data = imputer.fit_transform(app_train_data)
app_test_data = imputer.transform(app_test_data)

In [10]:
scaler = MinMaxScaler(feature_range = (0, 1))
app_train_data = scaler.fit_transform(app_train_data)
app_test_data = scaler.transform(app_test_data)

In [11]:
class Stat_Holder:
    
    def __init__(self, pca_component_no, classifier_name, classifier_params
                 , accurancy): 
        self.pca_component_no = pca_component_no
        self.classifier_name = classifier_name
        self.classifier_params = classifier_params
        self.accurancy = accurancy
        
    def __str__(self):
        return '{pca_component_no='+ str(self.pca_component_no) +\
                ' classifier_name=' + str(self.classifier_name) +\
                ' classifier_params=' + str(self.classifier_params) +\
                ' accurancy=' + str(self.accurancy) +\
                '}'
        
    def __lt__(self, other):

        
        # sort decending by accurancy
        return self.accurancy > other.accurancy 
# End of class Stat_Hlder   

def sort_print_stats(stats,top=10):    
    stats=sorted(stats)
    print_stats(stats,top)

def print_stats(stats,top=10):
    print('\n')
    print('{:^6}{:^12}{:^50}{:^10}{:^30}'
          .format('Sr. No.','No. of PCA','Classifier','Accuracy','Classifier Params'))
    print('_'*80)
    for index,stat in enumerate(stats[:top]):
        print('{:^6}{:^12}{:^50}{:>10.5}{:^30}'
              .format(index+1,stat.pca_component_no,stat.classifier_name
                      ,stat.accurancy,stat.classifier_params))

Now lets use various combinations of PCA and Random Forest to get baseline predictions

In [12]:
def get_estimater_name(clf):
    if type(clf).__name__ == 'GridSearchCV':
        estimater_name = 'GridSearchCV:'+ type(clf.estimator).__name__
    else:
        estimater_name = type(clf).__name__
    return estimater_name

def get_estimater_param(clf,clf_params):
    if type(clf).__name__ == 'GridSearchCV':
        for i,param in enumerate(clf_params.split(',')):
            if i > 0:
                parameters = parameters + ','
            parameters = param + '=' + str(clf.best_params_[param])    
    else:
        parameters = clf_params

    return parameters

def run_classifier(clf, train_data,train_labels,test_data,test_labels,pca_no,clf_params):
    clf.fit(train_data,train_labels)
    test_labels_predicted = clf.predict(test_data)
    #print(test_labels_predicted[:10])
    #print(test_labels[:10])
    accurancy = metrics.roc_auc_score(test_labels,test_labels_predicted)
   
    st = Stat_Holder(pca_no,get_estimater_name(clf),get_estimater_param(clf,clf_params),accurancy)
    return st

def run_classifiers(min_PCA, max_PCA, classifier_list):
    stats = []
    for classifier in classifier_list:
        clf = classifier.get('clf')
        clf_params = classifier.get('params')
        print('{} started with params {}'.format(get_estimater_name(clf),clf_params))
        #print(clf)
        
        #print(clf_params)
        if min_PCA is None:
            st = run_classifier(clf, app_train_data[:limit_data,],train_labels[:limit_data]
                                        ,app_test_data[:limit_data,],test_labels[:limit_data]
                                ,'NA',clf_params)
            print(st)
            stats.append(st)
        else:
            pca_range = range(min_PCA,max_PCA+1)
            for pca_no in pca_range:
                    print('\tPCA {} started'.format(pca_no), end=" ")
                    pca = PCA(n_components = pca_no,copy = True)
                    train_data_pca = pca.fit_transform(app_train_data) 
                    test_data_pca = pca.transform(app_test_data) 

                    st = run_classifier(clf, train_data_pca[:limit_data,],train_labels[:limit_data]
                                        ,test_data_pca[:limit_data,],test_labels[:limit_data]
                                        ,pca_no,clf_params)
                    stats.append(st)
                    print('\tPCA {} completed'.format(pca_no))
                    #print_stats([st])
        print('{} completed'.format(get_estimater_name(clf)))
    return stats
    


In [13]:
all_stats=[] #stats across all algo, never overridden
stats = [] #stats for specific group, overridden for each group

minPCA = 1
maxPCA = 5
limit_data = None
classifiers = []
classifiers.append({'clf':DecisionTreeClassifier(max_depth=5), 'params':'max_depth=5'} )
classifiers.append({'clf':KMeans(n_clusters=5), 'params':'n_clusters=5'} )
classifiers.append({'clf':GaussianMixture(n_components=3), 'params':'n_components=3'} )
classifiers.append({'clf':KNeighborsClassifier(n_neighbors=3), 'params':'n_neighbors=3'} )
#classifiers
stats=run_classifiers(minPCA,maxPCA,classifiers)
all_stats.extend(stats)

DecisionTreeClassifier started with params max_depth=5
	PCA 1 started 	PCA 1 completed
	PCA 2 started 	PCA 2 completed
	PCA 3 started 	PCA 3 completed
	PCA 4 started 	PCA 4 completed
	PCA 5 started 	PCA 5 completed
DecisionTreeClassifier completed
KMeans started with params n_clusters=5
	PCA 1 started 	PCA 1 completed
	PCA 2 started 	PCA 2 completed
	PCA 3 started 	PCA 3 completed
	PCA 4 started 	PCA 4 completed
	PCA 5 started 	PCA 5 completed
KMeans completed
GaussianMixture started with params n_components=3
	PCA 1 started 	PCA 1 completed
	PCA 2 started 	PCA 2 completed
	PCA 3 started 	PCA 3 completed
	PCA 4 started 	PCA 4 completed
	PCA 5 started 	PCA 5 completed
GaussianMixture completed
KNeighborsClassifier started with params n_neighbors=3
	PCA 1 started 	PCA 1 completed
	PCA 2 started 	PCA 2 completed
	PCA 3 started 	PCA 3 completed
	PCA 4 started 	PCA 4 completed
	PCA 5 started 	PCA 5 completed
KNeighborsClassifier completed


In [14]:
sort_print_stats(stats,top=15)



Sr. No. No. of PCA                     Classifier                     Accuracy       Classifier Params       
________________________________________________________________________________
  1        5                       GaussianMixture                     0.55948        n_components=3        
  2        4                       GaussianMixture                     0.54595        n_components=3        
  3        1                       GaussianMixture                     0.53848        n_components=3        
  4        5                     KNeighborsClassifier                  0.50986        n_neighbors=3         
  5        4                     KNeighborsClassifier                  0.50878        n_neighbors=3         
  6        3                     KNeighborsClassifier                  0.50432        n_neighbors=3         
  7        2                     KNeighborsClassifier                  0.50421        n_neighbors=3         
  8        1                            KMea

In [15]:
classifiers = []
classifiers.append({'clf':LogisticRegression(C = 0.1), 'params':'C = 0.1'} )
classifiers.append({'clf':LogisticRegression(C = 0.01), 'params':'C = 0.01'} )
classifiers.append({'clf':LogisticRegression(C = 0.001), 'params':'C = 0.001'} )
classifiers.append({'clf':LogisticRegression(C = 0.0001), 'params':'C = 0.0001'} )
classifiers.append({'clf':RandomForestClassifier(n_estimators = 50), 'params':'n_estimators = 50'} )
classifiers.append({'clf':RandomForestClassifier(n_estimators = 100), 'params':'n_estimators = 100'} )
classifiers.append({'clf':GradientBoostingClassifier(n_estimators = 100), 'params':'n_estimators = 100'} )

#stats=run_classifiers(None,None,classifiers)
#all_stats.extend(stats)
#sort_print_stats(stats)

In [16]:
# Summary of top 15 execution. 
sort_print_stats(all_stats,top=15)



Sr. No. No. of PCA                     Classifier                     Accuracy       Classifier Params       
________________________________________________________________________________
  1        5                       GaussianMixture                     0.55948        n_components=3        
  2        4                       GaussianMixture                     0.54595        n_components=3        
  3        1                       GaussianMixture                     0.53848        n_components=3        
  4        5                     KNeighborsClassifier                  0.50986        n_neighbors=3         
  5        4                     KNeighborsClassifier                  0.50878        n_neighbors=3         
  6        3                     KNeighborsClassifier                  0.50432        n_neighbors=3         
  7        2                     KNeighborsClassifier                  0.50421        n_neighbors=3         
  8        1                            KMea

In [17]:
limit_data = None
clusters = {'n_clusters':list(range(8,10))}
depths = {'max_depth':list(range(5,7))}
components = {'n_components':list(range(5,6))}
neighbors = {'n_neighbors':list(range(5,6))}

classifiers = []
classifiers.append({'clf':GridSearchCV(estimator=KMeans(), param_grid=clusters), 'params':'n_clusters'} )
classifiers.append({'clf':GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=depths), 'params':'max_depth'} )
#classifiers.append({'clf':GridSearchCV(estimator=GaussianMixture(), param_grid=components), 'params':'n_components'} )
#classifiers.append({'clf':GridSearchCV(estimator=KNeighborsClassifier(), param_grid=neighbors), 'params':'n_neighbors'} )
stats=run_classifiers(None,None,classifiers)
all_stats.extend(stats) 
sort_print_stats(stats)

GridSearchCV:KMeans started with params n_clusters
{pca_component_no=NA classifier_name=GridSearchCV:KMeans classifier_params=n_clusters=9 accurancy=0.5141859609460693}
GridSearchCV:KMeans completed
GridSearchCV:DecisionTreeClassifier started with params max_depth
{pca_component_no=NA classifier_name=GridSearchCV:DecisionTreeClassifier classifier_params=max_depth=5 accurancy=0.5046799772499094}
GridSearchCV:DecisionTreeClassifier completed


Sr. No. No. of PCA                     Classifier                     Accuracy       Classifier Params       
________________________________________________________________________________
  1        NA                    GridSearchCV:KMeans                   0.51419         n_clusters=9         
  2        NA            GridSearchCV:DecisionTreeClassifier           0.50468         max_depth=5          


In [18]:
limit_data = None
Cs = {'C':[.1,.01,.001,.0001]}
estimators = {'n_estimators':[50,100,150]}

classifiers = []
classifiers.append({'clf':GridSearchCV(estimator=LogisticRegression(), param_grid=Cs), 'params':'C'} )
#classifiers.append({'clf':GridSearchCV(estimator=RandomForestClassifier(), param_grid=estimators), 'params':'n_estimators'} )
#classifiers.append({'clf':GridSearchCV(estimator=GradientBoostingClassifier(), param_grid=estimators), 'params':'n_estimators'} )

stats=run_classifiers(None,None,classifiers)
all_stats.extend(stats)
sort_print_stats(stats)

GridSearchCV:LogisticRegression started with params C
{pca_component_no=NA classifier_name=GridSearchCV:LogisticRegression classifier_params=C=0.01 accurancy=0.5010428444042734}
GridSearchCV:LogisticRegression completed


Sr. No. No. of PCA                     Classifier                     Accuracy       Classifier Params       
________________________________________________________________________________
  1        NA              GridSearchCV:LogisticRegression             0.50104            C=0.01            


In [19]:
# Summary of top 15 execution. 
sort_print_stats(all_stats,top=15)



Sr. No. No. of PCA                     Classifier                     Accuracy       Classifier Params       
________________________________________________________________________________
  1        5                       GaussianMixture                     0.55948        n_components=3        
  2        4                       GaussianMixture                     0.54595        n_components=3        
  3        1                       GaussianMixture                     0.53848        n_components=3        
  4        NA                    GridSearchCV:KMeans                   0.51419         n_clusters=9         
  5        5                     KNeighborsClassifier                  0.50986        n_neighbors=3         
  6        4                     KNeighborsClassifier                  0.50878        n_neighbors=3         
  7        NA            GridSearchCV:DecisionTreeClassifier           0.50468         max_depth=5          
  8        3                     KNeighborsC

In [ ]:
class Acorn():

    def __init__(self, classifiers, X):
        self.classifiers = classifiers
        self.votes = [clf.predict(X) for clf in self.classifiers]
        self.election_results = None

    def _get_votes(self):
        # alter for flexiblity later
        if len(self.votes) == 3:
            return np.c_[self.votes[0],self.votes[1],self.votes[2]]
        if len(self.votes) == 5:
            return np.c_[self.votes[0],self.votes[1],self.votes[2],self.votes[3],self.votes[4]]
        else:
            print('number of classiefiers must be 3 or 5')

    def vote(self):
        vote_pool = self._get_votes()
        self.election_results = np.array([np.argmax(np.bincount(item)) for item in vote_pool])
        return self.election_results
    
    def spf_score(self, y, average='micro'):
        if np.shape(self.election_results) == np.shape(y):
            print('Score: {}\n'.format(np.sum(self.election_results == y)/np.shape(self.election_results)[0]))
            print('Precision:{} | Recall: {}\n'.format(precision_score(y, self.election_results, average=average),
                                                      recall_score(y, self.election_results, average=average)))
            print('F1 score: {}'.format(f1_score(y, self.election_results)))
    
    def roc_it(self, X, y, plot=False):
        p = np.array(None)
        d = 0
        for i, clf in enumerate(self.classifiers):
            if i == 0:
                p = clf.predict_proba(X)
            else:
                p = p + clf.predict_proba(X)
                d = i
        prob_pos = p/(d+1)
        prob_pos = np.array([prob_pos[i,x] for i, x in enumerate(np.argmax(prob_pos, axis=1))])
        
        #code adapted from matplotlib
        if plot == True:
            fpr, tpr, _ = roc_curve(y, prob_pos)
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, color='orange',
                    lw=1.5, label='ROC Curve (area = %0.2f)' % roc_auc)
            plt.plot([0,1], [0,1], color='steelblue', lw=1, linestyle='--')
            plt.xlim([0.0,1.0])
            plt.ylim([0.0,1.0])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Receiver Operating Characteristic')
            plt.legend(loc='lower right')
            plt.show()
        else:
            print('ROC_AUC score: {}'.format(roc_auc_score(y, prob_pos)))
        return p/(d+1)
    